# Vectors & word embeddings 
A workshop by UMN LATIS and the Libraries. 

Portions of the [Constellate's TAP Institute Introduction to Vector Databases and Semantic Searching](https://github.com/wjbmattingly/tap-2024-vector-databases?ref=cms-prod.constellate.org) and [the Carpentries' Text Analysis in Python](https://carpentries-incubator.github.io/python-text-analysis/) lessons are re-used in this lesson. Both lessons are licensed under [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/) as is the content presented below.  

## What we'll cover in this session
    - Representing words as numbers
    - TF-IDF
    - Word vectors
    - What are they? 
    - How they work + Benefits
    - Word Vectors with Spacy
    - Interpreting results
    - Word embeddings in 2d plots
    - Calculating distance between pairs
    - Semantic Search
        - Document embedding
    - Transformers


## Install required libraries
If you're working from your own machine you can use pip install to make sure you have downloaded all of the Python packages you'll need to use today. 

If you're working on notebooks.latis.umn.edu, there's no need to install any of these, since they're included in the virtual environment.

In [ ]:
# use !pip install to install libraries we'll use today
# numpy 2 does not work with sentence-transformers
!pip install spacy scikit-learn pandas matplotlib 'numpy<2' 

# or use conda
!conda install spacy scikit-learn pandas numpy matplotlib 'numpy<2'

# This command downloads the medium-sized English language model for spaCy.
# It uses the Python module-running option to run spaCy's download command for the "en_core_web_md" model.
!python -m spacy download en_core_web_md 

## Changing words into numbers
- Computers require numerical representations of things like images and words to be able to store, process, and manipulate them. 
- For most text analysis methods that will allows us to work with text as data, then, it's critical to use structured formats that represent words as numbers.
- At the base level computers already use encoding standards like Unicode to represent words as numbers, but Unicode represents specific characters in numerical form, not entire words. Since we're interested in working with the meanings of words and not characters, we'll use other methods to create numerical representations of our texts.
- Rather than manually assigning words to specific numbers on our own, we can utilize existing vector frameworks to transform our texts to numerical formats that allow us to analyze meaning.

### TF-IDF: Term Frequency, Inverse Document Frequency

- TF-IDF is a statistical measure using a matrix to evaluate how important a word is to a document or corpus. 
- In a TF-IDF matrix:
 - each row represents a document
 - each column represents a unique word
 - each cell contains a score for that word in the document
 
The score increases relative to the number of times that a word appears in a document, offset by its frequency in the entire corpus. So words that are more common across the entire corpus will have lower scores in a particular document, which helps us account for common terms. 

Let's use the spacy package to transform a collection of State of the Union addresses into a TF-IDF Document Term Matrix.

#### Import required libraries

In [1]:
import spacy
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Load the spaCy model
nlp = spacy.load("en_core_web_md")

In [ ]:
# create three simple documents
document = ['feed the duck', 'feed the goose', 'duck duck goose']

# Initialize the TfidfVectorizer
# Convert a collection of raw documents to a matrix of TF-IDF features.
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(document)

# Create a DataFrame for better readability
df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df

- Each row is a document (e.g., row 0 is "feed the duck"). 
- Each column is a word in our list of documents. 
- Each cell has a score for the word that increases relative to the number of times that a word appears in a document, offset by its frequency in the entire corpus.

#### Tokenize the SOTU corpus with spacy

In [5]:
# Collect all speech file paths
sotu = glob.glob("../sotu_text/*.txt")

def tokenize_speeches(speeches_glob, n):
    # Initialize a list to store preprocessed speeches
    processed_speeches = []
    
    # Process each speech
    for speech_path in speeches_glob[0:n]:
        with open(speech_path, 'r') as file:
            text = file.read()
    
            #tokenize each document using spacy
            doc = nlp(text)
            
            # Filter tokens and join them into a single string
            # ignore stop words, punctuation, spaces and digits
            tokens = [token.text.lower() for token in doc if not (token.is_stop or token.is_punct or token.is_space or token.is_digit or token.like_num)]
    
            # Re-join the list of tokens into a single string
            processed_text = ' '.join(tokens)
    
            # Append each cleaned and tokenized document to our list
            processed_speeches.append(processed_text)
            
    return processed_speeches

processed_speeches = tokenize_speeches(sotu, 5)

In [6]:
# look at the first 250 characters of the first speech, tokenized:
processed_speeches[0][0:250]

'fellow citizens senate house representatives benignant providence almighty god representatives states people brought deliberate public good gratitude nation sovereign arbiter human events commensurate boundless blessings enjoy peace plenty contentmen'

### Convert to a TF-IDF Vector


In [ ]:
# Initialize the TfidfVectorizer
# Convert a collection of raw documents to a matrix of TF-IDF features.
vectorizer = TfidfVectorizer()

# Fit and transform the first 5 preprocessed speeches to a TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(processed_speeches)

You can type in the name of the class and Shift+Tab to get a pop up with the documentation for `TfidfVectorizer`.

In [ ]:
TfidfVectorizer

In [ ]:
# get the matrix shape: 5 rows, by 5,770 columns
# each row is a document, each column is a token
tfidf_matrix.shape

#### Use Pandas to view the matrix

In [ ]:
# Create a DataFrame for better readability
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Display the DataFrame
df_tfidf.head()

In [ ]:
# Number of top words to extract for each document
top_n = 5

# Find the top n words for each document
top_words_per_document = []
for index, row in df_tfidf.iterrows():
    top_words = row.sort_values(ascending=False).head(top_n).index.tolist()
    top_words_per_document.append(top_words)

# Print the top words for each document
for doc_index, words in enumerate(top_words_per_document):
    print(f"Document {doc_index + 1} top words:\n {words}\n")

In TF-IDF the words are assigned numbers based on their importance to each document and the corpus, but the numbers don't actually tell us anything about a word's meaning or its relationship to other words. Words vectors give us a way to understand more about how words in a corpus relate to each other.

## Word Vectors

When people refer to word vectors, or word embeddings, they're talking about a way of representing each word as a numerical vector in a high-dimensional space, typically from 50 to 300 dimensions. 

Key properties of word vectors:

1. Dimensionality: In TF-IDF there as many dimensions as there are words in the vocabulary. Word vectors typically have fewer dimensions (e.g., 100 or 300).

2. Density: Unlike TF-IDF where a vector consists mostly of 0s, word vectors are dense, meaning most elements are non-zero.

3. Learned from data: Word vectors are typically learned from large text corpora using machine learning techniques. They capture semantic and syntactic relationships between words based on their usage patterns in the text.

4. Semantic relationships: Similar words have similar vectors. For example, the vectors for "king" and "queen" might be close to each other.

5. Arithmetic operations: Word vectors often exhibit interesting arithmetic properties. A classic example is: vector("king") - vector("man") + vector("woman") ≈ vector("queen").

## How Word Vectors Work

Word vectors work on the principle of distributional semantics, which states that words that occur in similar contexts tend to have similar meanings. Machine learning models analyze large amounts of text data to learn these representations.

There are different popular models out there, such as Word2Vec, developed by Google. We'll stick with spaCy, and use the vector model that comes with "en_core_web_md".

For more information about how these models produce meaningful word embeddings, and how the models are trained, see the Carpentries' lesson on [the Word2Vec algorithm](https://carpentries-incubator.github.io/python-text-analysis/08-wordEmbed_word2vec-algorithm/index.html). 

In [ ]:
token = nlp("freedom")
vector = token.vector

#the vector has 1 row and 300 columns, for each dimension
vector.shape

The vector itself isn't very interesting.

In [ ]:
df = pd.DataFrame(vector.reshape(-1, len(vector)))
df

### Cosine Similarity
But we can compare different word vectors to measure their similarity, in terms of the mathematical differences between the vectors. We'll first use spaCy's similarity method.

It’s important to note that the effectiveness of the similarity measurement depends on the quality of the model's embeddings and whether the model has been trained on a relevant corpus for the task at hand.

In [ ]:
def compare_words(word1, word2):
    similarity = nlp(word1).similarity(nlp(word2))
    print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")

In [ ]:
compare_words('king', 'president')
compare_words('king', 'pauper')
compare_words('president', 'pauper')

What does this tell us? It means that semantically and syntactically, king and president are used in more similar ways than king and pauper This is because the embeddings produced are the result of an embedding model that saw a lot of English texts and in those texts, kings and presidents are represented in similar ways, as you would expect. 

### Show word embeddings on a 2d plot

In [ ]:
words = ['nurse', 'doctor', 'farmer', 'athlete', 'librarian', 'teacher']

# create a vector for each word in a list called vectors
vectors = [nlp(word).vector for word in words]

# Reduce the dimensions of the vectors using Principal component analysis (PCA)
# We are choosing 2 compenents so we can plot them in a 2D space
pca = PCA(n_components = 2)
vectors_2d = pca.fit_transform(vectors)

In [ ]:
# Plot the words in 2D space
plt.figure(figsize=(12, 8))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c='b', alpha=0.7)

for i, word in enumerate(words):
    plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), xytext=(5, 2), 
                 textcoords='offset points', ha='right', va='bottom',
                 bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                 arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

plt.title("Word Embeddings in 2D Space")
plt.xlabel("First Principal Component")
plt.ylabel("Second Principal Component")
plt.grid(True)
plt.show()

#plt.savefig('../outputs/word_embeddings_2d.svg', format='svg')
#print("Plot saved as 'word_embeddings_2d.svg'")

This visualization helps us understand how word embeddings capture semantic relationships. Words with similar meanings or uses tend to cluster together in the vector space, while words with different meanings are farther apart. Even in this simplified 2D representation, we can see how the embedding space organizes words in a way that reflects their semantic relationships.

Remember, though, that this is a significant simplification of the original high-dimensional space. In the full embedding space, these relationships are even more nuanced and accurate.

### Euclidean Distance

The Euclidian distance formula makes use of the Pythagorean theorem, where $a^2 + b^2 = c^2$. We can draw a triangle between two points, and calculate the hypotenuse to find the distance. This distance formula works in two dimensions, but can also be generalized over as many dimensions as we want. 

In [ ]:
# Calculate and print the Euclidean distances between pairs
print("\nDistances between pairs:")
for i in range(len(words)):
    for j in range(i+1, len(words)):
        distance = np.linalg.norm(vectors_2d[i] - vectors_2d[j])
        print(f"{words[i]} - {words[j]}: {distance:.4f}")

## Semantic search & vector databases
In this section we will create a vector database with many State of the Union documents, and query it semantically. 

### What are Vector Databases?
Vector databases are specialized storage and retrieval systems designed to handle high-dimensional vector data efficiently. Unlike traditional databases that work with structured data like numbers and text, vector databases are optimized for managing and querying vector embeddings - numerical representations of data points in a multi-dimensional space.

At their core, vector databases address the challenge of similarity search in large datasets. They excel at finding the most similar items to a given query, which is crucial for applications like recommendation systems, image recognition, and natural language processing. For instance, in a vector database storing product information, you could easily find similar products based on various attributes, all encoded as vectors.

The key advantage of vector databases lies in their ability to perform fast approximate nearest neighbor (ANN) searches. Traditional databases might struggle with the "curse of dimensionality" when dealing with high-dimensional data, but vector databases employ specialized indexing techniques to maintain efficiency. This makes them particularly useful for AI and machine learning applications, where data is often represented in high-dimensional vector spaces.

For those new to the concept, you can think of a vector database as a system that organizes information in a way that mirrors how our brains associate related concepts. Just as we can quickly recall words or images that are similar to a given prompt, vector databases can rapidly retrieve data points that are "close" to each other in a mathematical sense. This capability opens up exciting possibilities for creating more intelligent and intuitive data-driven applications across various domains.

In [ ]:
# Using !pip install
#!pip install annoy txtai sentence-transformers rank-bm25

# or conda
#!conda install python-annoy txtai sentence-transformers
!pip install rank-bm25

In [4]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch
from annoy import AnnoyIndex
from rank_bm25 import BM25Okapi

Let's load all of the sotu speeches into an unprocessed and untokenized list. 

In [10]:
speeches = []
for speech in sotu:
    with open(speech, 'r') as file:
        speeches.append(file.read())
print(len(speeches))        

236


### Vectorize with sentence-transformers
Now that we have all our documents, it comes time to vectorize them, or convert them into a sequence of vectors. This is where we pass the texts to a machine learning model and capture the output vector for each of them. To do this, though, we need a model loaded. We will be using the sentence-transformers library. It makes this process as simple as possible with only two lines of code. First, we will need to load the model.

In [11]:
# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # This is a standard, efficient model

Now that we have loaded our model we can (optionally) specify the specific device. The code below will put it onto your GPU, if available. If you don't know if this is enabled on your device, then it likely is not. The steps to activate cuda are very specific and require you to install certain packages in a certain way. If you do not have cuda, then the default will be the cpu. With 236 documents, this will not be an issue.

In [12]:
# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Let's encode the speeches! We can do that with a single line. I like to set show_progress_bar to True. This allows me to see how long things take on my machine.

In [13]:
embeddings = model.encode(speeches, show_progress_bar=True)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(speeches)
df.columns = ['speeches']
df["embedding"] = list(embeddings)
df

,speeches,embedding
0,\n\n Fellow-Citizens of the Senate and of the ...,"[0.006504625, -0.0010013514, 0.009295043, -0.0..."
1,\n\n Fellow-Citizens of the Senate and House o...,"[-0.020296935, -0.019036775, 0.08283116, -0.03..."
2,\n\n Fellow Citizens of the Senate and of the ...,"[-0.07354095, 0.06733189, 0.05767685, -0.03805..."
3,\n\n To the Senate and House of Representative...,"[-0.06475066, 0.064519495, 0.06715353, -0.0519..."
4,\n\n To the Congress of the United States: \n\...,"[-0.05823529, 0.022737347, 0.026741277, -0.026..."
...,...,...
231,\n\n Fellow-Citizens of the Senate and House o...,"[-0.0370819, 0.055570964, 0.0578656, -0.012479..."
232,\n\n Fellow Citizens of the Senate and of the ...,"[0.041680336, -0.07579356, 0.116802275, -0.068..."
233,\n\n Fellow-Citizens of the Senate and House o...,"[-0.061863095, 0.039531752, 0.069955446, -0.02..."
234,\n\n Fellow-Citizens of the Senate and House o...,"[-0.01136333, 0.027026301, 0.016789565, -0.037..."


At this stage, we now have all the data necessary to create a vector database! 

### BM25
BM25 (Best Matching 25) is a ranking function used in information retrieval systems, particularly in search engines. It's an advanced form of TF-IDF (Term Frequency-Inverse Document Frequency) that provides a way to rank documents based on their relevance to a given search query.

BM25 improves upon simpler ranking methods by incorporating document length normalization. This means it can account for the fact that longer documents are more likely to contain a given term simply due to their length, rather than because of relevance.

The algorithm calculates a score for each document based on the query terms it contains. It considers both how often a term appears in a document (term frequency) and how rare the term is across all documents (inverse document frequency). However, it also applies a saturation function to prevent common terms from dominating the score.

For those new to information retrieval, BM25 can be thought of as a way of determining which documents in a collection are most relevant to a user's search query. It's widely used in practice due to its effectiveness and relatively simple implementation.

Now, let's look at how we can implement BM25 in Python using a DataFrame's 'speeches' field.

To do that, we first need to tokenize our text. We could use `nlp()` to use our spaCy model from earlier, but that would be quite time consuming. To save time let's just split our documents up by whitespace. 

In [20]:
tokenized_docs = []
for doc in df["speeches"]:
    split_doc = doc.split()
    # to use a slower but better tokenizer using spaCy:
    # split_doc = nlp(doc)
    tokenized_docs.append(split_doc)

Our BM250kapi tool expects each document to be a list of tokens though, and our current speeches are strings. So let's convert them to make processing easier.  

This single line of code will create a BM25 index for us! At this stage, we have essentially created a search engine index. 

In [21]:
bm25_index = BM25Okapi(tokenized_docs)

To query it, we need to create a query, tokenize the query, and then use the `get_scores()` method to retrieve the results. It's also best practice to sort these based on the scores. Finally, we can retrieve the results from the original DataFrame. In the cell below, we will have all the code necessary to perform these operations. I've done this so that you can more easily test this out with multiple queries. To understand what's happening in the code, though, let's breakdown each step here.

In [34]:
query = "Soviet Union"
tokenized_query = query.split()
doc_scores = bm25_index.get_scores(tokenized_query)
ranked_docs = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)[:5]

for idx, score in ranked_docs:
    print(f"Score: {score:.4f} - Speech {idx}\n {df['speeches'][idx][0:250]}")
    print("-------------", "\n")

Score: 5.3509 - Speech 100
 

Mr. President, Mr. Speaker, Members of the 96th Congress, fellow citizens: 

This last few months has not been an easy time for any of us. As we meet tonight, it has never been more clear that the state of our Union depends on the state of the worl
------------- 

Score: 5.0278 - Speech 51
 Mr. President, Mr. Speaker, Members of the Congress: 

This 82d Congress faces as grave a task as any Congress in the history of our Republic. The actions you take will be watched by the whole world. These actions will measure the ability of a free p
------------- 

Score: 4.9385 - Speech 97
 

To the Congress of the United States: 

I have the honor to report to the Congress on the state of the Union. 

This is the eighth such report that, as President, I have been privileged to present to you and to the country. On previous occasions, i
------------- 

Score: 4.9246 - Speech 128
 

Mr. President, Mr. Speaker, Members of the 96th Congress, and my fellow citizens:Tonigh

- `query = "Soviet"`: This line defines the search query. 
- `tokenized_query = query.split()`: This line tokenizes the query string. The split() method without arguments splits the string on whitespace, creating a list of individual words. For our simple query, this results in ["Soviet", "Union"]. 
- `doc_scores = bm25_index.get_scores(tokenized_query)` - Use the pre-computed BM25 index (bm25_index) to score each document in the corpus based on the tokenized query.
- `get_scores()` method calculates a relevance score for each document in relation to the query.
- The result `doc_scores` is a list of floating-point numbers, where each number represents the relevance score of the corresponding document in the original corpus.

### More information
- [Transformers and LLMs](https://carpentries-incubator.github.io/python-text-analysis/10-finetuning-transformers/index.html)
- [The Word2Vec algorithm](https://carpentries-incubator.github.io/python-text-analysis/08-wordEmbed_word2vec-algorithm/index.html)